# Photometric Extinction Coefficients

In order to calculate magnitudes above the atmosphere, we need to estimate the atmosphere effect on the flux of the objects. To do that, we have to perform aperture photometry in the same objects and in all the individual exposures, calculate the magnitudes and then, compare them with the airmass of that exposure. After that, we fit a linear regresion model to the data sample, where the variables are:

\begin{equation*}
    \begin{split}
        X &= \text{AIRMASS}\\
        Y &= \text{INSTRUMENTAL MAG}
    \end{split}
\end{equation*}

Finally, we calculate the $b_0$ term of the regression model. This value is the $k$ extinction coefficient, that corresponds to the slope of the linear fit in a AIRMASS v/s INS_MAG plot. This value has to be calculated for every CCD, but we can calculate the mean $K$ extinction coefficient per filter. This values are:

\begin{equation*}
    \begin{split}
    K_Y &= 0.1228\\
    K_u &= 0.3913
    \end{split}
\end{equation*}

in units of $[mag/airmass]$. Also, the mean airmass for each filter are:

\begin{align*}
    \bar{X_Y} &= 1.405\\
    \bar{X_u} &= 1.061
\end{align*}

This notebook perform all this work and calculates all the $k$ extinction coefficients for each CCD and the mean $K$ extinction coefficient for each filter.

In [1]:
from astropy.io import fits #to load FITS images
import time #count time
from photutils import CircularAperture,aperture_photometry #to create apertures and perform photometry
import numpy as np #numpy, to open files and maths
from sklearn.linear_model import LinearRegression #apply a linear regression to data
import matplotlib.pyplot as plt #in case of plotting the data
import os #to change directories

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Extinction Coefficient Calculator

In [2]:
def extinction_coeff(decam_filter,ccd):
    os.chdir('/home/alex-cfrd/Escritorio/extinction_coeff/filter_'+decam_filter+'/'+ccd+'/') #move to directory
    if decam_filter=='Y':
        airmass = np.array([2.01,1.78,1.68,1.58,1.5,1.43,1.37,1.31,1.26,1.11,1.14,1.17,1.21,1.25,1.29,1.34,1.4,1.47]).reshape((-1,1)) #airmass of each exposure
        n = 18 #18 exposures in filter Y
    elif decam_filter=='u':
        airmass = np.array([1.22,1.18,1.15,1.12,1.09,1.07,1.06,1.04,1.03,1.02,1.01,1.01,1.01,1.01,1.01,1.01,1.02,1.03,1.04,1.05,1.07,1.09]).reshape((-1,1)) #airmass of each exposure
        n = 22 #22 exposures in filter u
    coord_0 = np.genfromtxt('coord_0.txt') #object coord in first exposure
    ext_coeff = []
    for j in range(1,32):
        if j==30 and ccd=='N':
            continue
        obj_center = [] #list of object center in each exposure
        offsets = np.genfromtxt('offsets_'+ccd+str(j)+'.txt') #offsets of exposures, to calculate the other coords
        for i in range(n):
            x = coord_0[j-1][0] - offsets[i][0] #x coord
            y = coord_0[j-1][1] - offsets[i][1] #y coord
            obj_center.append([x,y]) #fill coords list
        apertures = CircularAperture(obj_center,r=10) #convert coords to apertures
        MAGS = [] #empty list of magnitudes
        for i in range(1,n+1): #from exposure 1 to 18
            if i<10:
                image = fits.open('back'+ccd+'_'+decam_filter+'0'+str(i)+'.fits')
            else:
                image = fits.open('back'+ccd+'_'+decam_filter+str(i)+'.fits')
            data = image[j].data #data of CCD [j]
            phot = aperture_photometry(data,apertures[i-1]) #perform photometry
            mag = -2.5*np.log10(phot['aperture_sum']) #calculate magnitude
            MAGS.append(mag[0]) #append data to mag list
        MAGS = np.array(MAGS) #convert mag list to tarray
        model = LinearRegression() #set a linear regression model
        model.fit(airmass,MAGS) #adjust linearly airmass and magnitudes
        b_0 = round(float(model.coef_),5) #extract b_0 coeff = k extinction coeff
        ext_coeff.append(b_0) #extinction coeffs from 1 to S31
        if j<10:
            print(str(b_0)[:8]+',')#'k extinction coefficient CCD ['+ccd+str(j)+']  =',b_0) #print each k
        else:
            print(str(b_0)[:8]+',')#'k extinction coefficient CCD ['+ccd+str(j)+'] =',b_0) #print each k
    #STATISTICS
    print('---------------------------------------------')
    print('STATISTICS')
    mean = str(np.mean(ext_coeff)) #mean of CCDs ext_coeff
    if ccd=='S':
        ind_med = ' ['+ccd+str(ext_coeff.index(np.median(ext_coeff))+1)+']' #calculate median ext_coeff
    median = str(np.median(ext_coeff)) #calculate median ext_coeff
    minimum,ind_min = str(np.min(ext_coeff)),' ['+ccd+str(ext_coeff.index(np.min(ext_coeff))+1)+']' #extract min ext_coeff
    maximum,ind_max = str(np.max(ext_coeff)),' ['+ccd+str(ext_coeff.index(np.max(ext_coeff))+1)+']' #extract max ext_coeff
    std = str(np.std(ext_coeff)) #calculate ext_coeff standard deviation
    suma = str(sum(ext_coeff))
    print('---------------------------------------------')
    print('SIGMA Mean   =',mean[:9])
    if ccd=='S':
        print('SIGMA Median =',median[:9]+ind_med)
    elif ccd=='N':
        print('SIGMA Median =',median[:9])
    print('SIGMA Min    =',minimum[:9]+ind_min)
    print('SIGMA Max    =',maximum[:9]+ind_max)
    print('SIGMA Std    =',std[:9])
    print('SIGMA Sum    =',suma[:9])

## Filter $Y$ [S1-S31]

In [3]:
extinction_coeff('Y','S')

0.11727,
0.11793,
0.11332,
0.12294,
0.1228,
0.12252,
0.1323,
0.11989,
0.11985,
0.1197,
0.12237,
0.13256,
0.13355,
0.11604,
0.11933,
0.12519,
0.12409,
0.11976,
0.12539,
0.11726,
0.11884,
0.12472,
0.12118,
0.13073,
0.12033,
0.12216,
0.12398,
0.13307,
0.13389,
0.12899,
0.12842,
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 0.1235603
SIGMA Median = 0.12252 [S6]
SIGMA Min    = 0.11332 [S3]
SIGMA Max    = 0.13389 [S29]
SIGMA Std    = 0.0055569
SIGMA Sum    = 3.83037


## Filter $Y$ [N1-N31] (no [N30])

In [4]:
extinction_coeff('Y','N')

0.12093,
0.11962,
0.12072,
0.1221,
0.12424,
0.13321,
0.12978,
0.12089,
0.11679,
0.1194,
0.12013,
0.12361,
0.12626,
0.11631,
0.12238,
0.12284,
0.12328,
0.12547,
0.12888,
0.11798,
0.11965,
0.11414,
0.12835,
0.1266,
0.11744,
0.1266,
0.12686,
0.12688,
0.12001,
0.13064,
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 0.1230663
SIGMA Median = 0.12261
SIGMA Min    = 0.11414 [N22]
SIGMA Max    = 0.13321 [N6]
SIGMA Std    = 0.0045902
SIGMA Sum    = 3.6919900


## Filter $u$ [S1-S31]

In [5]:
extinction_coeff('u','S')

0.32098,
0.36529,
0.35716,
0.36523,
0.41083,
0.41817,
0.54112,
0.37353,
0.38323,
0.38054,
0.38698,
0.41049,
0.40152,
0.38117,
0.38258,
0.39302,
0.39297,
0.43238,
0.44422,
0.38496,
0.39936,
0.3956,
0.40568,
0.42826,
0.36452,
0.39505,
0.39532,
0.40979,
0.40412,
0.40595,
0.41053,
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 0.3980822
SIGMA Median = 0.39532 [S27]
SIGMA Min    = 0.32098 [S1]
SIGMA Max    = 0.54112 [S7]
SIGMA Std    = 0.0353713
SIGMA Sum    = 12.34055


## Filter $u$ [N1-N31] (no [N30])

In [6]:
extinction_coeff('u','N')

0.36922,
0.37772,
0.32226,
0.39862,
0.3903,
0.41462,
0.42095,
0.39511,
0.33896,
0.36022,
0.37846,
0.45277,
0.42454,
0.38172,
0.36483,
0.34736,
0.37087,
0.38664,
0.42679,
0.34806,
0.403,
0.37795,
0.38346,
0.41392,
0.31706,
0.40766,
0.36068,
0.37759,
0.38399,
0.49174,
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 0.3862356
SIGMA Median = 0.38259
SIGMA Min    = 0.31706 [N25]
SIGMA Max    = 0.49174 [N30]
SIGMA Std    = 0.0361206
SIGMA Sum    = 11.58707
